In [ ]:
%pip install scikit-learn 
%pip install pandas
%pip install xgboost
%pip install optuna
%pip install lightgbm
%pip install matplotlib
%pip install catboost
%pip install plotly
%pip install haversine
%pip install umap-learn
%pip install reverse_geocoder
%pip install shapely

In [208]:
import pandas as pd
import numpy as np
import optuna as op
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objs as go

from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.decomposition import PCA
from umap import UMAP
from sklearn.cluster import KMeans
from haversine import haversine
import reverse_geocoder as rg
from shapely.geometry import LineString, Point

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.metrics import mean_squared_error

In [303]:
# Gathering and combining training and testing data (But keeping the target column on 'y' for training.) 
training_df = pd.read_csv('./train.csv', index_col='id')
testing_df = pd.read_csv('./test.csv', index_col='id')
# Drop the three outlier rows from the training set
to_drop = [20991, 34363, 33228]
training_df = training_df.drop(to_drop)
# Split the target variable from the training set
y = training_df.MedHouseVal
training_df = training_df.drop(['MedHouseVal'], axis=1)
# Concatenate the training and testing sets
X = pd.concat([training_df, testing_df], axis=0, ignore_index=True)

# X is both training and testing data! Will unmerge them after feature engineering, so I can tune, train, and fit the models on the training data only!
# This was done so I could easily feature engineer both the training and testing data at the same time.

                                                            Feature Engineering

Clustering

In [304]:
coordinates = X[['Latitude', 'Longitude']].values
clustering = KMeans(n_clusters=20, max_iter=500, random_state=42).fit(coordinates)
cluster_centers = {i: tuple(centroid) for i, centroid in enumerate(clustering.cluster_centers_)}

def cluster_features(df):
    for i, cc in enumerate(cluster_centers.values()):
        df[f'cluster_{i}'] = df.apply(lambda x: haversine((x['Latitude'], x['Longitude']), cc, unit='ft'), axis=1)
    return df

X = cluster_features(X)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Coordinates with PCA and UMAP

In [305]:
pca = PCA().fit(coordinates)
X['pca_lat'] = pca.transform(coordinates)[:,0]
X['pca_lon'] = pca.transform(coordinates)[:,1]

umap = UMAP(n_components=2, 
            n_neighbors=50, 
            random_state=42).fit(coordinates)
X['umap_lat'] = umap.transform(coordinates)[:,0]
X['umap_lon'] = umap.transform(coordinates)[:,1]

Cartesian coordinates rotation

In [306]:
X['rot_15_x'] = (np.cos(np.radians(15)) * X['Longitude']) + \
                  (np.sin(np.radians(15)) * X['Latitude'])
    
X['rot_15_y'] = (np.cos(np.radians(15)) * X['Latitude']) + \
                  (np.sin(np.radians(15)) * X['Longitude'])
    
X['rot_30_x'] = (np.cos(np.radians(30)) * X['Longitude']) + \
                  (np.sin(np.radians(30)) * X['Latitude'])
    
X['rot_30_y'] = (np.cos(np.radians(30)) * X['Latitude']) + \
                  (np.sin(np.radians(30)) * X['Longitude'])

Finding location of cities (and coastline)

In [307]:
# Extracting county info for each location based on its coordinates, and encoding that info as a numerical value in 'X'.
coordinates = list(zip(X['Latitude'], X['Longitude']))
results = rg.search(coordinates)
X['place'] = [x['admin2'] for x in results]

places = ['Los Angeles County', 'Orange County', 'Kern County',
          'Alameda County', 'San Francisco County', 'Ventura County',
          'Santa Clara County', 'Fresno County', 'Santa Barbara County',
          'Contra Costa County', 'Yolo County', 'Monterey County',
          'Riverside County', 'Napa County']

def replace(x):
    if x in places:
        return x
    else:
        return 'Other'
    
X['place'] = X['place'].apply(lambda x: replace(x))
le = LabelEncoder()
X['place'] = le.fit_transform(X['place'])

Getting the distance to cities and coast points

In [308]:
# Manually gathered location of major cities
SC = (38.576931, -121.494949)
SF = (37.780080, -122.420160)
SJ = (37.334789, -121.888138)
LA = (34.052235, -118.243683)
SD = (32.715759, -117.163818)
# Finds distance from coordinates in dataframe from cities.
X['dist_SC'] = X.apply(lambda x: haversine((x['Latitude'], x['Longitude']), SC, unit='ft'), axis=1)
X['dist_SF'] = X.apply(lambda x: haversine((x['Latitude'], x['Longitude']), SF, unit='ft'), axis=1)
X['dist_SJ'] = X.apply(lambda x: haversine((x['Latitude'], x['Longitude']), SJ, unit='ft'), axis=1)
X['dist_LA'] = X.apply(lambda x: haversine((x['Latitude'], x['Longitude']), LA, unit='ft'), axis=1)
X['dist_SD'] = X.apply(lambda x: haversine((x['Latitude'], x['Longitude']), SD, unit='ft'), axis=1)
X['dist_nearest_city'] = X[['dist_SC', 'dist_SF', 'dist_SJ', 
                              'dist_LA', 'dist_SD']].min(axis=1)

# Manually gathered location of coastline
coast_points = LineString([(32.664, -117.161), (33.206, -117.383),
                           (33.777, -118.202), (34.463, -120.014),
                           (35.427, -120.881), (35.928, -121.489),
                           (36.982, -122.028), (37.611, -122.491),
                           (38.355, -123.060), (39.792, -123.821),
                           (40.799, -124.188), (41.755, -124.197)])
# Finds shortest distance to coast from coordinates
X['dist_to_coast'] = X.apply(lambda x: Point(x['Latitude'], x['Longitude']).distance(coast_points), axis=1)

In [331]:
# Splitting df back
n_train = len(training_df)
training_df = X.iloc[:n_train, :]
testing_df = X.iloc[n_train:, :]
y = y.reset_index(drop=True)
testing_df = testing_df.reset_index(drop=True)
# Fix the index of testing_df to start from 37137
testing_df.index += 37137

                                                        Hyperparameter Tuning

In [ ]:
# Used to tune hyperparameters of LGBM model
class LGBMObjective:
    def __init__(self, model, X, y):
        self.model = model
        self.X = X
        self.y = y
    
    def __call__(self, trial):
        # Define the search space for the hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 500, 1400)
        max_depth = trial.suggest_int('max_depth', 7, 9)
        learning_rate = trial.suggest_float('learning_rate', 0.018, 0.038, log=True)
        num_leaves = trial.suggest_int('num_leaves', 60, 180)
        min_child_samples = trial.suggest_int('min_child_samples', 70, 140)

        # Set the hyperparameters in the model
        self.model.set_params(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            num_leaves=num_leaves,
            min_child_samples=min_child_samples
        )

        # Calculate the root mean squared error (RMSE) using cross-validation
        cv = KFold(n_splits=8, shuffle=True, random_state=42)
        rmse_scores = np.sqrt(-1 * cross_val_score(self.model, self.X, self.y, cv=cv, scoring='neg_mean_squared_error', n_jobs=1))
        mean_rmse = rmse_scores.mean()

        return mean_rmse

lgbm = LGBMRegressor(n_jobs=2)
obj = LGBMObjective(lgbm, training_df, y)

# Create the Optuna study and optimize the hyperparameters
study = op.create_study(direction='minimize')
study.optimize(obj, n_trials=15)

# Print the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

In [ ]:
# Used to tune hyperparameters of LGBM model
class CatBoostObjective:
    def __init__(self, model, X, y):
        self.model = model
        self.X = X
        self.y = y
    
    def __call__(self, trial):
        # Define the search space for the hyperparameters
        iterations = trial.suggest_int('iterations', 20, 100)
        depth = trial.suggest_int('depth', 6, 10)
        #learning_rate = trial.suggest_int('learning_rate', 0.1, 1)

        # Set the hyperparameters in the model
        self.model.set_params(
            iterations=iterations,
            depth=depth
            #learning_rate=learning_rate
        )

        # Calculate the root mean squared error (RMSE) using cross-validation
        cv = KFold(n_splits=8, shuffle=True, random_state=42)
        rmse_scores = np.sqrt(-1 * cross_val_score(self.model, self.X, self.y, cv=cv, scoring='neg_mean_squared_error', n_jobs=1))
        mean_rmse = rmse_scores.mean()

        return mean_rmse

catB = CatBoostRegressor(thread_count=4)
obj = CatBoostObjective(catB, training_df, y)

# Create the Optuna study and optimize the hyperparameters
study = op.create_study(direction='minimize')
study.optimize(obj, n_trials=15)

# Print the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

In [313]:
models = {
    'LGBM Regressor': LGBMRegressor(n_estimators=800, max_depth=7, learning_rate=0.028, num_leaves=100, min_child_samples=100),
    'XGBoost': XGBRegressor(n_estimators=400, max_depth=6, learning_rate=0.01),
    'CatBoost': CatBoostRegressor(iterations=72, depth=6, learning_rate=0.475, thread_count=4)
}

# Split your data into training and validation sets
train_X, validation_X, train_y, validation_y = train_test_split(training_df, y, train_size=0.8, test_size=0.2, random_state=42)

In [ ]:
# Individual Base Model predictions
for model_name, model in models.items():
    cv_scores = cross_val_score(model, train_X, train_y, cv=8, scoring='neg_root_mean_squared_error')
    mean_rmse = -np.mean(cv_scores)
    print(f"RMSE for the {model_name} model with cross-validation: {mean_rmse}")

                                                                Stacking

In [ ]:
# Fit each of your base models on the full training data.
for model_name, model in models.items():
    model.fit(train_X, train_y)

In [321]:
# Make predictions using the base models on the validation set
base_model_predictions = []
for model_name, model in models.items():
    prediction = model.predict(validation_X)
    base_model_predictions.append(prediction)

# Stack predictions horizontally (i.e., column-wise)
stacked_predictions = np.column_stack(base_model_predictions)

In [324]:
# Train a meta-model using the stacked predictions as input and the true validation_y values as output. linear regressor as the meta-model is fine.
meta_model = LGBMRegressor()
meta_model.fit(stacked_predictions, validation_y)

LGBMRegressor()

In [325]:
# Make predictions using the meta-model on the validation set
validation_meta_model_predictions = meta_model.predict(stacked_predictions)

# Calculate the RMSE
validation_rmse = np.sqrt(mean_squared_error(validation_y, validation_meta_model_predictions))
print(f"Meta model RMSE on validation data: {validation_rmse}")

Meta model RMSE on validation data: 0.5126655757952071


In [332]:
# Make predictions using the base models on the test set
test_base_model_predictions = []
for model_name, model in models.items():
    test_prediction = model.predict(testing_df)
    test_base_model_predictions.append(test_prediction)

# Stack predictions horizontally (i.e., column-wise)
test_stacked_predictions = np.column_stack(test_base_model_predictions)

# Make predictions using the meta-model on the test set
test_meta_model_predictions = meta_model.predict(test_stacked_predictions)

# Create a DataFrame for the submission
submission = pd.DataFrame({
    'id': testing_df.index,
    'MedHouseVal': test_meta_model_predictions
})

# Save the submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

In [ ]:
print(submission)

                                                    EDA & initial data visualization

Heatmap of MedHouseVal based on location (from initial training data)

In [ ]:

fig = px.scatter(training_df, x='Latitude', y='Longitude', color='MedHouseVal',
                 color_continuous_scale='Jet', size_max=25, opacity=1,
                 title='Target Distribution', width=800, height=800)

fig.update_xaxes(title_text='Latitude', showgrid=False, showline=True)
fig.update_yaxes(title_text='Longitude', showgrid=False, showline=True)
fig.show()

Displays feature values

In [ ]:
# Initialize a subplot grid with 3 columns and 3 rows.
fig = sp.make_subplots(rows=3, cols=3, subplot_titles=training_df.columns, horizontal_spacing=0.05, vertical_spacing=0.1)
# Loop through the columns and create a histogram for each feature.
for i, col in enumerate(training_df.columns):
    row = i // 3 + 1
    col_idx = i % 3 + 1
    x = training_df[col]
    if col in ['AveBedrms', 'Population', 'AveOccup', 'AveRooms']:
        x = np.log(x)
    fig.add_trace(go.Histogram(x=x, nbinsx=15, marker=dict(color='#000080')), row=row, col=col_idx)

fig.update_layout(title_text='Features Distribution', title_x=0.5, showlegend=False, width=1000, height=800, plot_bgcolor='#D3D3D3', paper_bgcolor='#FFFDD0',
                  font=dict(family='Arial', color='#000080'))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False, showticklabels=False)
fig.show()

for column in training_df.columns:
    print(f"Statistics for {column}:")
    print(f"Min: {training_df[column].min()}")
    print(f"Max: {training_df[column].max()}")
    print(f"Mean: {training_df[column].mean()}")
    print(f"Std: {training_df[column].std()}")
    print("\n")

Plot of the PCA and UMAP features

In [ ]:
# Combine 'X' and 'y' into a single DataFrame
data_for_plt = X.copy()
data_for_plt['MedHouseVal'] = y

# Create a subplot grid with 1 row and 2 columns
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=['PCA', 'UMAP'], horizontal_spacing=0.1)

# Create the PCA scatter plot
fig.add_trace(
    go.Scatter(x=data_for_plt['pca_lat'], y=data_for_plt['pca_lon'], mode='markers',
               marker=dict(size=6, color=data_for_plt['MedHouseVal'], colorscale='Jet', opacity=1), showlegend=False),
    row=1, col=1
)

# Create the UMAP scatter plot
fig.add_trace(
    go.Scatter(x=data_for_plt['umap_lat'], y=data_for_plt['umap_lon'], mode='markers',
               marker=dict(size=6, color=data_for_plt['MedHouseVal'], colorscale='Jet', opacity=1), showlegend=False),
    row=1, col=2
)

# Update the layout of the subplot grid
fig.update_layout(title_text='PCA and UMAP', title_x=0.5, width=1200, height=600)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

# Show the plot
fig.show()

Plot of the Cartesian coordinates rotated

In [ ]:
# Combine 'X' and 'y' into a single DataFrame
data_for_plt = X.copy()
data_for_plt['MedHouseVal'] = y

# Create a subplot grid with 1 row and 2 columns
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=['15 degrees rotation', '30 degrees rotation'], horizontal_spacing=0.1)

# Create the 15 degrees rotation scatter plot
fig.add_trace(
    go.Scatter(x=data_for_plt['rot_15_x'], y=data_for_plt['rot_15_y'], mode='markers',
               marker=dict(size=6, color=data_for_plt['MedHouseVal'], colorscale='Jet', opacity=0.5), showlegend=False),
    row=1, col=1
)

# Create the 30 degrees rotation scatter plot
fig.add_trace(
    go.Scatter(x=data_for_plt['rot_30_x'], y=data_for_plt['rot_30_y'], mode='markers',
               marker=dict(size=6, color=data_for_plt['MedHouseVal'], colorscale='Jet', opacity=0.5), showlegend=False),
    row=1, col=2
)

# Update the layout of the subplot grid
fig.update_layout(title_text='Cartesian Coordinates Rotation', title_x=0.5, width=1200, height=600)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

# Show the plot
fig.show()

Plot of feature importances for LGBM model

In [ ]:
lgbm_model = models['LGBM Regressor']

# Get feature importances from the LGBM model
feature_importances = lgbm_model.feature_importances_

# Create a DataFrame with feature names and their importance values
feature_importance_df = pd.DataFrame({'feature': training_df.columns, 'importance': feature_importances})

# Sort the DataFrame by importance values in descending order
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)

# Create a Plotly bar chart
fig = go.Figure(go.Bar(x=feature_importance_df['importance'], y=feature_importance_df['feature'], orientation='h'))

# Customize the chart's appearance
fig.update_layout(
    title='Feature Importances',
    xaxis_title='Importance',
    yaxis_title='Feature',
    font=dict(
        family="Calibri",
        size=14,
        color="#444444"
    )
)

# Show the chart
fig.show()